In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import zipfile
import os
import time
import shutil
import math

In [2]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [3]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add
path="smaller_pcaps"


# WIRESHARK FEATURES dst,src and Protocol added

In [ ]:
name_list=find_the_way('./FE','_FE.csv')
for i in tqdm(name_list):
    df=pd.read_csv(i)
    WS_s_d=i.replace("_FE.csv","_WS.csv")
    s_d=pd.read_csv(WS_s_d)
    df["WS_src"]=s_d["_ws.col.Source"]
    df["WS_dst"]=s_d["_ws.col.Destination"]
    df["Protocol"]=s_d["_ws.col.Protocol"]
    y=df["Label"]
    del df["Label"]
    df["Label"]=y
    df.to_csv(i,index=None) 
    flag_fixer(i)
    os.remove(WS_s_d)





# NEW FEATURES: dst_IP_diversity, dst_port_diversity, src_IP_diversity IP_add_count

In [15]:
for i in name_list:  
    print(i)
    dst_IP_diversity=[]
    dst_port_diversity=[]
    src_IP_diversity=[]
    src_IP_v_dst_IP={}
    
    IP_add_count=[]
    src_MAC_v_dst_IP={}
    
    src_IP_v_dst_port={}
    dst_IP_v_src_port={}
    df=pd.read_csv(i)
    count=0
    for j in tqdm(range(len(df))):
        
        if df['IP_src'][j] in src_IP_v_dst_IP:
            if df['IP_dst'][j] not in src_IP_v_dst_IP[df['IP_src'][j]]:
                src_IP_v_dst_IP[df['IP_src'][j]].append(df['IP_dst'][j])
        else:
            src_IP_v_dst_IP[df['IP_src'][j]]=[]
            src_IP_v_dst_IP[df['IP_src'][j]].append(df['IP_dst'][j])
        dst_IP_diversity.append(len(src_IP_v_dst_IP[df['IP_src'][j]]))
        
        ###############################################################################
        
        if df['IP_dst'][j] in dst_IP_v_src_port:
            if df['IP_src'][j] not in dst_IP_v_src_port[df['IP_dst'][j]]:
                dst_IP_v_src_port[df['IP_dst'][j]].append(df['IP_src'][j])
        else:
            dst_IP_v_src_port[df['IP_dst'][j]]=[]
            dst_IP_v_src_port[df['IP_dst'][j]].append(df['IP_src'][j])
        src_IP_diversity.append(len(dst_IP_v_src_port[df['IP_dst'][j]]))
        
        ###############################################################################
        if df['IP_src'][j] in src_IP_v_dst_port:
            if df['dport_bare'][j] not in src_IP_v_dst_port[df['IP_src'][j]]:
                src_IP_v_dst_port[df['IP_src'][j]].append(df['dport_bare'][j])
        else:
            src_IP_v_dst_port[df['IP_src'][j]]=[]
            src_IP_v_dst_port[df['IP_src'][j]].append(df['dport_bare'][j])
        dst_port_diversity.append(len(src_IP_v_dst_port[df['IP_src'][j]]))
        #################################################################
        if df['Ether_src'][j] in src_MAC_v_dst_IP:
            if df['IP_dst'][j] not in src_MAC_v_dst_IP[df['Ether_src'][j]]:
                src_MAC_v_dst_IP[df['Ether_src'][j]].append(df['IP_dst'][j])
        else:
            src_MAC_v_dst_IP[df['Ether_src'][j]]=[]
            src_MAC_v_dst_IP[df['Ether_src'][j]].append(df['IP_dst'][j])
        IP_add_count.append(len(src_MAC_v_dst_IP[df['Ether_src'][j]]))
        count+=1
        #if count>300000:break
    y=df["Label"]
    del df["Label"]
    df["dst_IP_diversity"]=dst_IP_diversity
    df["dst_port_diversity"]=dst_port_diversity
    df["src_IP_diversity"]=src_IP_diversity
    df["IP_add_count"]=IP_add_count
    df["Label"]=y
    df.to_csv(i,index=None) 


./FE\ARP_MitM_x_FE.csv


100%|███████████████████████████████████████████████████████████████████████| 400000/400000 [00:15<00:00, 25448.46it/s]


# Sliding Windows Features

In [5]:
files_add=find_the_way('./FE','FE.csv')
files_add

['./FE\\ARP_MitM_x_FE.csv']

In [6]:
def diff(x):
    x = pd.Series(x)
    result=x.diff()
    result=result.fillna(0)
    return result
def mean(x,WS):
    x = pd.Series(x)
    result=x.rolling(WS).mean()
    result=result.fillna(result[WS-1])
    return result
def std(x,WS):
    x = pd.Series(x)
    result=x.rolling(WS).std()
    result=result.fillna(result[WS-1])
    return result

def mean_EW(x):
    x = pd.Series(x)
    result=x.expanding(min_periods=1).mean()
    return result
def std_EW(x):
    x = pd.Series(x)
    result=x.expanding(min_periods=1).std()
    result=result.fillna(0)
    return result
def sum_of_EW(x):
    x = pd.Series(x)
    result=x.expanding(min_periods=1).sum()
    return result

def var(x,WS):
    x = pd.Series(x)
    result=x.rolling(WS).var()
    result=result.fillna(result[WS-1])
    return result
def flagsum(x):
    total_f=0
    result=[]
    for i in x:
        if i==1:
            total_f+=1
        result.append(total_f)
    return result
def portsum(x):
    total_ports=[]
    result=[]
    for i in x:
        if i not in total_ports:
            total_ports.append(i)
        result.append(len(total_ports))
    return result     

In [7]:
n=['ts',"WS_src","WS_dst",
 'pck_size',
 #'Ether_dst',
 #'Ether_src','IP_src',
 #'IP_dst',
   'UDP_sport',
 'UDP_dport','TCP_sport',
 'TCP_dport',
#'IP_src',
 #'IP_dst', 
'TCP_FIN',
 'TCP_SYN',
 'TCP_RST',
 'TCP_PSH',
 'TCP_ACK',
 'TCP_URG',
 'TCP_ECE',
 'TCP_CWR',
 'TCP_window',"payload_bytes","entropy","Label"]

In [8]:
for j in files_add:
    df=pd.read_csv(j)
    break

In [9]:
outputfolder="SW"
folder(outputfolder)

for windows_size in range(2,4):
    for j in files_add:
        print(j[5:])
        df=pd.read_csv(j)#,usecols=n)

        df["sport"]=df["UDP_sport"]+df["TCP_sport"]
        df["dport"]=df["UDP_dport"]+df["TCP_dport"]

        del df["UDP_sport"]
        del df["TCP_sport"]
        del df["UDP_dport"]
        del df["TCP_dport"]

        df.to_csv("temp.csv")


        df=pd.read_csv("temp.csv")
        #df["ID"] = df["Ether_src"] +"@" +df["IP_src"] +"====>"+df["Ether_dst"]+ "@"+ df["IP_dst"]
        df["ID"] = df["WS_src"]+"=>"+df["WS_dst"]
        #del df["Ether_src"] 
        #del df["IP_src"] 
        #del df["Ether_dst"] 
        #del df["IP_dst"]
        del df["WS_src"] 
        del df["WS_dst"]

        cols=[ 'TCP_FIN',
     'TCP_SYN',
     'TCP_RST',
     'TCP_PSH',
     'TCP_ACK',
     'TCP_URG',
     'TCP_ECE',
     'TCP_CWR', 'Label', 'ID', 'pck_size_diff', 'pck_size_mean',
               'pck_size_std',  'ts_diff', 'ts_mean', 'ts_std',
                'TCP_window_diff', 'TCP_window_mean', 'TCP_window_std',"payload_bytes_diff","payload_bytes_mean","payload_bytes_std",
                'entropy_diff',
         'entropy_mean',
         'entropy_std',
'pck_size_mean_EW','pck_size_std_EW',
'ts_mean_EW', 'ts_std_EW',
'TCP_window_mean_EW','TCP_window_std_EW',
'payload_bytes_mean_EW','payload_bytes_std_EW',
'entropy_mean_EW','entropy_std_EW',
'ts_sum_of_EW', 'pck_size_sum_of_EW', 
'payload_bytes_sum_of_EW','entropy_sum_of_EW','TCP_window_sum_of_EW',"Label"]

        IDS=sorted(list(df[df.columns[-1]].unique()))
        func_sw=[mean,std] #functions
        func_ew=[diff,mean_EW,std_EW,sum_of_EW] #functions   
        func_name_sw=["mean","std"]
        func_name_ew=["diff","mean_WE","std_WE","sum_of_EW"]
        
        f_list=["pck_size","ts",'TCP_window',"payload_bytes","entropy"]#, "dport","sport"     'TCP_SYN', 'TCP_ACK'    'TCP_FIN', 'TCP_SYN', 'TCP_RST', 'TCP_PSH', 'TCP_ACK', 'TCP_URG', 'TCP_ECE', 'TCP_CWR',  
        fark=[#'pck_size_diff',
         'pck_size_mean',
         'pck_size_std',
         #'ts_diff',
         'ts_mean',
         'ts_std',
         #'TCP_window_diff',
         'TCP_window_mean',
         'TCP_window_std',#"payload_bytes_diff",
          "payload_bytes_mean","payload_bytes_std",
         #'entropy_diff',
         'entropy_mean',
         'entropy_std']
        WS=windows_size
        flag=1
        for i in tqdm(IDS):
            smaller = df[df["ID"]==i]
            smaller=smaller.reset_index()
            del smaller["index"]
            # smaller["ts"]=np.array(diff(smaller["ts"].values)) older way
            #print( smaller["ts"])
            smaller["ts"]=smaller["ts"].values-smaller["ts"].values[0]
            #print( smaller["ts"])
            for ii in f_list:
                for jjj,iii in enumerate (func_ew):
                    name=str(ii)+"_"+str(func_name_ew[jjj])
                    smaller[name]=iii(smaller[ii])
            if len(smaller)>=WS:
                for ii in f_list:
                    for jjj,iii in enumerate (func_sw):
                        name=str(ii)+"_"+str(func_name_sw[jjj])
                        smaller[name]=iii(smaller[ii],WS)
            else:
                for ii in fark:
                        smaller[ii]=0
            smaller["dport_sum"]=portsum(smaller["dport"].values)
            smaller["sport_sum"] =portsum(smaller["sport"].values)
            smaller['TCP_FIN_sum'  ]=flagsum (smaller[ 'TCP_FIN'  ].values)
            smaller['TCP_SYN_sum'  ]=flagsum (smaller['TCP_SYN'  ].values)
            smaller['TCP_RST_sum'  ]=flagsum (smaller['TCP_RST'  ].values)
            smaller['TCP_PSH_sum'  ]=flagsum (smaller['TCP_PSH'  ].values)
            smaller['TCP_ACK_sum'  ]=flagsum (smaller['TCP_ACK'  ].values)
            smaller['TCP_URG_sum'  ]=flagsum (smaller['TCP_URG'  ].values)
            smaller['TCP_ECE_sum'  ]=flagsum (smaller['TCP_ECE'  ].values)
            smaller['TCP_CWR_sum'  ]=flagsum (smaller['TCP_CWR'  ].values)
            #del smaller["dport"]        
            #del smaller["sport"]        
            #del smaller["ID"]
            

            smaller["TCP_FIN_ratio"]=smaller["TCP_FIN"]/(smaller["TCP_FIN_sum"]+10e-20)
            smaller["TCP_SYN_ratio"]=smaller["TCP_SYN"]/(smaller["TCP_SYN_sum"]+10e-20)
            smaller["TCP_RST_ratio"]=smaller["TCP_RST"]/(smaller["TCP_RST_sum"]+10e-20)
            smaller["TCP_PSH_ratio"]=smaller["TCP_PSH"]/(smaller["TCP_PSH_sum"]+10e-20)
            smaller["TCP_ACK_ratio"]=smaller["TCP_ACK"]/(smaller["TCP_ACK_sum"]+10e-20)
            smaller["TCP_URG_ratio"]=smaller["TCP_URG"]/(smaller["TCP_URG_sum"]+10e-20)
            smaller["TCP_ECE_ratio"]=smaller["TCP_ECE"]/(smaller["TCP_ECE_sum"]+10e-20)
            smaller["TCP_CWR_ratio"]=smaller["TCP_CWR"]/(smaller["TCP_CWR_sum"]+10e-20)
            col_list= ["TCP_FIN_sum","TCP_SYN_sum","TCP_RST_sum","TCP_PSH_sum","TCP_ACK_sum","TCP_URG_sum","TCP_ECE_sum","TCP_CWR_sum"]
            smaller['sum'] = smaller[col_list].sum(axis=1)
            smaller["TCP_FIN_SR"]=smaller["TCP_FIN"]/(smaller["sum"]+10e-20)
            smaller["TCP_SYN_SR"]=smaller["TCP_SYN"]/(smaller["sum"]+10e-20)
            smaller["TCP_RST_SR"]=smaller["TCP_RST"]/(smaller["sum"]+10e-20)
            smaller["TCP_PSH_SR"]=smaller["TCP_PSH"]/(smaller["sum"]+10e-20)
            smaller["TCP_ACK_SR"]=smaller["TCP_ACK"]/(smaller["sum"]+10e-20)
            smaller["TCP_URG_SR"]=smaller["TCP_URG"]/(smaller["sum"]+10e-20)
            smaller["TCP_ECE_SR"]=smaller["TCP_ECE"]/(smaller["sum"]+10e-20)
            smaller["TCP_CWR_SR"]=smaller["TCP_CWR"]/(smaller["sum"]+10e-20)

            if flag:
                smaller.to_csv(j[5:],header=True,index=False)
                flag=0
            else:
                smaller.to_csv(j[5:],header=False,index=False,mode="a")
        df=pd.read_csv(j[5:])
        features=df.columns
        count=[]
        bos=[]

        #for ii,i in enumerate (features):
            #a=pd.DataFrame(df.groupby(i).size())
            #if a.shape[0]>1:
                #print(a)
               # count.append(i)
                #print("________________________________________________________________\n\n")
            #else:
                #bos.append(i)
        df=df.sort_values(by='Unnamed: 0')

        y=df["Label"]
        #for i in bos:
            #print(i)
            #del df[i]
        del df["Label"]
        del df['Unnamed: 0']
        df["Label"]=y
        last_name=f'./{outputfolder}/last_{str(WS)}_{j[5:]}'
        df.to_csv(last_name,index=None)
        print(WS,"-------------------------------------------------------------------------------")


ARP_MitM_x_FE.csv


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:14<00:00,  1.42s/it]
C:\Users\kahra\AppData\Local\Temp\ipykernel_20024\1465388484.py:132: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(j[5:])


2 -------------------------------------------------------------------------------
ARP_MitM_x_FE.csv


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:15<00:00,  1.51s/it]
C:\Users\kahra\AppData\Local\Temp\ipykernel_20024\1465388484.py:132: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(j[5:])


3 -------------------------------------------------------------------------------
